In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, sys
import functools

os.chdir('..')
sys.path.append('..')
from data import dataset
from models import burn_in_lstm

/home/yuki/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yuki/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yuki/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yuki/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
def load_model():
    model = burn_in_lstm.SimpleSequentialLSTM()
    # model = burn_in_lstm.BurnInStateLSTM()
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    adam = tf.keras.optimizers.Adam(lr=1e-3)
    metrics = [
        tf.keras.metrics.BinaryAccuracy(name='acc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    model.compile(loss=loss, optimizer=adam, metrics=metrics)
    model.build(input_shape=[(None, 14, 20), (None, 19), (None, 3)])

    model_basedir = '/home/yuki/Documents/ymx/stock_research/train'
    if 'label.h5' in os.listdir(model_basedir):
        model.load_weights(os.path.join(model_basedir, 'label.h5'))
        print(' [*] Loaded pretrained model label.h5')
    return model

model = load_model()

 [*] Loaded pretrained model label.h5


In [9]:
import pprint

stock_basics = pd.read_csv('./data/stock_basics.csv')

def get_codenum_representation():
    value_set = list(set(stock_basics['codenum'].values.tolist()))
    value_set = list(map(lambda x: str(x).zfill(6), value_set))
    value_hash = dict(zip(value_set, list(range(len(value_set)))))
    return value_hash

stock_code_hash = get_codenum_representation()
stock_code_hash_reversed = dict(zip(stock_code_hash.values(), stock_code_hash.keys()))
# pprint.pprint(stock_code_hash)
print('OK')

OK


In [17]:
import heapq

def from_code_to_name(codestr):
    codenum = stock_code_hash_reversed[int(codestr)]
    dfline = stock_basics[stock_basics['codenum'] == int(codenum)]
    assert len(dfline) == 1, f'codestr={codestr} len(dfline)={len(dfline)}'
    return dfline['name'].values[0]

def get_knn_mapping(k=7):
    heapdict = {key: [] for key in stock_code_hash.keys()}
    niter = 0
    for x, xcode in stock_code_hash.items():
        ycodes = list(stock_code_hash.values()).copy()
        ycodes.remove(stock_code_hash[x])
        xcodes_arr = np.array([xcode] * len(ycodes), dtype=np.int32)
        ycodes_arr = np.array(ycodes, dtype=np.int32)
        simarray = model.stock_cosine_similarity(xcodes_arr, ycodes_arr)
        topk = tf.argsort(simarray, direction='DESCENDING')[: k]
        topk = topk.numpy().tolist()
        for elem in topk:
            heapdict[x].append(from_code_to_name(ycodes[elem]))
        if niter % 100 == 1:
            print('\r', f'>>>>>>>>>> iter={niter}', end='')
        niter += 1
    print('')
    return heapdict


stock_knn_mapping = get_knn_mapping()
print(stock_knn_mapping['600000'])

 >>>>>>>>>> iter=3801
['国海证券', '东北证券', '永太科技', '国光电器', '吉药控股', '安阳钢铁', '鹏欣资源']


In [24]:
print(stock_knn_mapping['601021'])

['芯源微', '天宇股份', '先达股份', '快意电梯', '新华保险', '豫园股份', '民和股份']
